<a href="https://colab.research.google.com/github/dalozgar/AO/blob/main/AG3_DAVID_ANGEL_LOZANO_GARCIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AG3- Actividad Guiada 3<br>
Nombre: DAVID ANGEL LOZANO GARCIA

Link: https://colab.research.google.com/drive/1KOBkO-WIKPx1w8XrLH1k3x3-4x_bcKsw

Github: https://github.com/dalozgar/AO

#Carga de librerias

In [1]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

#Carga de los datos del problema

In [2]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion : 
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file) 

#Coordendas 51-city problem (Christofides/Eilon)
#file = "eil51.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/eil51.tsp", file)

#Coordenadas - 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp" ; urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/att48.tsp", file)




('swiss42.tsp', <http.client.HTTPMessage at 0x7f514eb63c10>)

In [3]:
#Modulos extras, no esenciales
import numpy as np
import matplotlib.pyplot as plt
import imageio                    #Para construir las imagenes con gif
from google.colab import files    #Para descargar ficheros generados con google colab

from tempfile import mkstemp      #Para genera carpetas y ficheros temporales

import random                     #Para generar valores aleatorios


In [4]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())



In [5]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
                
#dir(problem)

15

#Funciones basicas


In [6]:
 
#Funciones basicas
###############################################################################      
     
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion 

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)


 

#Busqueda Aleatoria

In [7]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  Nodos = list(problem.get_nodes())
  
  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto
  
  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)
    
    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia
      
      
  print("Mejor solución:" , mejor_solucion) 
  print("Distancia     :" , mejor_distancia) 
  return mejor_solucion  

    
#Busqueda aleatoria con 5000 iteraciones    
solucion = busqueda_aleatoria(problem, 5000)    
 

Mejor solución: [0, 39, 12, 26, 36, 35, 31, 17, 41, 21, 24, 9, 25, 28, 37, 20, 34, 40, 30, 10, 38, 22, 27, 32, 33, 13, 23, 8, 7, 6, 19, 16, 15, 3, 1, 18, 4, 29, 14, 11, 2, 5]
Distancia     : 3622


#Busqueda Local  2-opt

In [8]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################

def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):
      
      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]  

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)         

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))
 

nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))

 

Distancia Solucion Incial: 3622
Distancia Mejor Solucion Local: 3393


# Generación alternativa de soluciones vecinas

## 1. Rotación
En primer lugar, he programado la siguiente función, genera_vecina_2, que realiza una selección aleatoria de elementos y los rota circularmente, hasta devolver la mejor solución encontrada mediante este método.
Tras realizar diversas pruebas, compruebo que esta función no obtiene muy buenos resultados y que su comportamiento es bastante cercano a la búsqueda aleatoria, aunque la voy a conservar porque la voy a utilizar más adelante.



In [20]:
import copy

def genera_vecina_2(solucion):
  #Generador de soluciones vecinas mediante rotación circular de un conjunto aleatorio de nodos.
  
  mejor_solucion = []
  mejor_distancia = 10e100

  n_alterados = random.randint(2, len(solucion)-1) #numero de nodos que voy a reordenar
  

  mask = np.zeros(len(solucion))
  mask[:n_alterados]  = 1
  np.random.shuffle(mask)

  # print (f"solucion = {solucion}")
  # print (f"n_alterados  = {n_alterados}")
  # print (f"mascara  = {mask}")
  
  seleccion = [ solucion[i] for i in range(len(solucion)) if mask[i]==1 ]
  
  # print (f"seleccion  = {seleccion}")

  for desplazamiento in range (0, n_alterados):
    
    vecina = copy.deepcopy(solucion)
    copia = copy.deepcopy(seleccion)
    copia2 = copy.deepcopy(seleccion)
    
    for i in range (len(seleccion)): 
      copia2[(i+desplazamiento) %len(seleccion)] = copia[i]

    for i in range (len(solucion)-1,-1,-1):
      if (mask[i]==1):
        vecina[i] = copia2.pop()
      
    #Se evalua la nueva solución ...
    distancia_vecina = distancia_total(vecina, problem)         

    #... para guardarla si mejora las anteriores
    if distancia_vecina <= mejor_distancia:
      mejor_distancia = distancia_vecina
      mejor_solucion = vecina

  return mejor_solucion



# solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))
 

nueva_solucion = genera_vecina_2(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))


Distancia Solucion Incial: 3622
Distancia Mejor Solucion Local: 3557


## 2. Opt-3
La siguiente función de búsqueda de vecindades obtiene la mejor de todas las reestructuraciones de la solución que se le pasa como parámetro de entrada, obtenidas mediante permutación de sus elementos tomados de tres en tres.

Da buenos resultados de búsqueda, pero es demasiado lenta al comprobar muchas combinaciones.

In [10]:
import copy
import itertools

def genera_vecina_3(solucion):
  #Generador de soluciones vecinas: 3-opt (intercambiar 2 nodos) 
  #Si hay N nodos se generan (N-2)x(N-3)x(N-4)x6 soluciones
  #print(solucion)
  
  mejor_solucion = []
  mejor_distancia = 10e100

  for mask1 in range (0, len(solucion)-2 ):
    for mask2 in range (mask1+1, len(solucion)-1):
      for mask3 in range (mask2+1, len(solucion)):

        mask = np.zeros(len(solucion))
        mask[mask1] = 1
        mask[mask2] = 1
        mask[mask3] = 1

        seleccion = [ solucion[i] for i in range(len(solucion)) if mask[i]==1 ]

        permutaciones = itertools.permutations(seleccion,3)

        for perm in permutaciones:
          # print (f"Permutacion: {perm}")
          vecina = copy.deepcopy(solucion)
          list_perm = list(perm)

          for i in range (len(solucion)-1,-1,-1):
            if (mask[i]==1):
              vecina[i] = list_perm.pop()

          #Se evalua la nueva solución ...
          distancia_vecina = distancia_total(vecina, problem)         

          #... para guardarla si mejora las anteriores
          if distancia_vecina <= mejor_distancia:
            mejor_distancia = distancia_vecina
            mejor_solucion = vecina


 
  return mejor_solucion
      

# solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))
 

nueva_solucion = genera_vecina_3(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))




Distancia Solucion Incial: 3622
Distancia Mejor Solucion Local: 3289


# Búsqueda en entornos variables

He realizado algunas modificaciones en el algoritmo de búsqueda local propuesto para añadir aleatoriedad variando el entorno, basándome en el documento:
https://jamoreno.webs.ull.es/www/papers/VNS2PL.pdf
En particular, se ha aplicado BVNS (Basic Variable Neighbourhood Search), página 5.

De esta forma se añade al algoritmo la capacidad de "escapar" de óptimos locales. Por las pruebas realizadas se deduce que no es tarea fácil y conforme nos vamos acercando al mínimo va resultando cada vez más difícil encontrar iterativamente mejores soluciones.
Tras realizar varias pruebas, la opción que mejor resultado me ha dado ha sido combinar las funciones genera_vecina y genera_vecina_2. La función genera_vecina_3 es capaz de "profundizar" bastante y obtiene buenos resultados en pocas iteraciones, pero tiene el inconveniente de que tarda mucho en cada iteración.




In [11]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem, k_max):
  mejor_solucion = []
  
  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  k = 0                   # Contador de iteraciones de BVNS
  # f = genera_vecina
  while(1):
    iteracion +=1         #Incrementamos el contador
    k +=1
    print(f"# iter= {iteracion} ... k = {k}... dist = {mejor_distancia}")

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina_2(solucion_referencia) # Agitación
    vecina = genera_vecina (vecina) #Búsqueda local

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)
 
    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia 
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina
      k = 0
      print (f"Reseteando k")
    
    
    if k == k_max:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion) 
      print("Distancia     :" , mejor_distancia) 
      return mejor_solucion
      
    solucion_referencia = vecina
 

k_max = 1000
sol = busqueda_local(problem, k_max )  


# iter= 1 ... k = 1... dist = 4847
Reseteando k
# iter= 2 ... k = 1... dist = 4390
Reseteando k
# iter= 3 ... k = 1... dist = 3758
Reseteando k
# iter= 4 ... k = 1... dist = 3519
Reseteando k
# iter= 5 ... k = 1... dist = 3188
Reseteando k
# iter= 6 ... k = 1... dist = 2922
Reseteando k
# iter= 7 ... k = 1... dist = 2756
Reseteando k
# iter= 8 ... k = 1... dist = 2602
Reseteando k
# iter= 9 ... k = 1... dist = 2488
Reseteando k
# iter= 10 ... k = 1... dist = 2406
Reseteando k
# iter= 11 ... k = 1... dist = 2336
Reseteando k
# iter= 12 ... k = 1... dist = 2267
Reseteando k
# iter= 13 ... k = 1... dist = 2227
Reseteando k
# iter= 14 ... k = 1... dist = 2188
Reseteando k
# iter= 15 ... k = 1... dist = 2153
Reseteando k
# iter= 16 ... k = 1... dist = 2079
Reseteando k
# iter= 17 ... k = 1... dist = 2049
Reseteando k
# iter= 18 ... k = 1... dist = 2019
Reseteando k
# iter= 19 ... k = 1... dist = 1992
Reseteando k
# iter= 20 ... k = 1... dist = 1970
Reseteando k
# iter= 21 ... k = 1... dist 

#Simulated Annealing

In [12]:
###############################################################################
# SIMULATED ANNEALING
###############################################################################

#Generador de 1 solucion vecina 2-opt 100% aleatoria (intercambiar 2 nodos)
#Mejorable eligiendo otra forma de elegir una vecina.
def genera_vecina_aleatorio(solucion):

  #Se eligen dos nodos aleatoriamente
  i,j = sorted(random.sample( range(1,len(solucion)) , 2))
  
  #Devuelve una nueva solución pero intercambiando los dos nodos elegidos al azar
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
  
 
sol_aleatoria = genera_vecina_aleatorio(solucion) 


In [13]:

#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
  if random.random() <  math.exp( -1*d / T)  :
    return True
  else:
    return False

#Funcion de descenso de temperatura
def bajar_temperatura(T):
  return T*0.99
 



# MÉTODO DE RECOCIDO SIMULADO CON MEJORAS:

Se ha modificado el propuesto en clase, cambiando la forma de bucar nuevas soluciones candidatas. En este caso, se ha reaprovechado lo ya aplicado en apartados anteriores. En concreto, se creará una solución candidata de forma aleatoria, pero se le hará una búsqueda local mediante 2-opt antes de seguir procesando. De este modo, se tendrán mejores candidatos en cada iteración, mejorando las opciones de encontrar buenas soluciones.

In [15]:
def recocido_simulado(problem, temperatura ):
  #problem = datos del problema
  #T = Temperatura
  
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  
  N=0
  while temperatura > .0001:
    N+=1
    print (f"N= {N} - temp = {temperatura} distancia = {mejor_distancia}")
    #Genera una solución vecina
    vecina =genera_vecina_aleatorio(solucion_referencia) #Agitación 1
    vecina =genera_vecina_2(vecina) #Agitación 2- Búsqueda
    vecina = genera_vecina(vecina) #Búsqueda local 

    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
      
    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
    #Si la nueva vecina es mejor se cambia  
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(temperatura, abs(distancia_referencia - distancia_vecina) ) :
      #solucion_referencia = copy.deepcopy(vecina)
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    #Bajamos la temperatura
    temperatura = bajar_temperatura(temperatura)
 
  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

sol  = recocido_simulado(problem, 100000)

 



N= 1 - temp = 100000 distancia = 1e+101
N= 2 - temp = 99000.0 distancia = 3842
N= 3 - temp = 98010.0 distancia = 3542
N= 4 - temp = 97029.9 distancia = 3346
N= 5 - temp = 96059.601 distancia = 3183
N= 6 - temp = 95099.00499 distancia = 3022
N= 7 - temp = 94148.0149401 distancia = 2963
N= 8 - temp = 93206.53479069899 distancia = 2959
N= 9 - temp = 92274.469442792 distancia = 2853
N= 10 - temp = 91351.72474836408 distancia = 2853
N= 11 - temp = 90438.20750088044 distancia = 2853
N= 12 - temp = 89533.82542587163 distancia = 2853
N= 13 - temp = 88638.48717161291 distancia = 2853
N= 14 - temp = 87752.10229989678 distancia = 2853
N= 15 - temp = 86874.5812768978 distancia = 2853
N= 16 - temp = 86005.83546412883 distancia = 2728
N= 17 - temp = 85145.77710948754 distancia = 2728
N= 18 - temp = 84294.31933839266 distancia = 2695
N= 19 - temp = 83451.37614500873 distancia = 2640
N= 20 - temp = 82616.86238355865 distancia = 2640
N= 21 - temp = 81790.69375972306 distancia = 2624
N= 22 - temp = 8097